In [ ]:
! pip install opencv-python == 4.5.5.64

In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

In [ ]:
# import the necessary packages
from loader_util.projects.tiny_imagenet import config
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import SimplePreprocessor
from loader_util.preprocessing import MeanPreprocessor
from loader_util.callbacks import EpochCheckpoint
from loader_util.callbacks import TrainingMonitor
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import DeeperGooglenet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import argparse
import json

In [ ]:
# script constants
output_path = config.output_path
start_epoch = 0
model = None

In [ ]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=18, zoom_range=0.15,
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest")

# load the RGB means for the training set
means = json.loads(open(config.rgb_mean_path).read())

In [ ]:
# initialize the image preprocessors
sp = SimplePreprocessor(64, 64)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

# initialize the training and validation dataset generators
trainGen = HDF5DatasetGenerator(config.train_hdf5_path,
                                batch_size=64,
                                aug=aug,
                                preprocessors=[sp, mp, iap])
valGen = HDF5DatasetGenerator(config.val_hdf5_path,
                              batch_size=64,
                              preprocessors=[sp, mp, iap], )

In [ ]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = DeeperGooglenet.build(width=64, height=64, depth=3,
                                  classes=config.num_classes, reg=0.0002)
    opt = Adam(1e-3)
    model.compile(loss="categorical_crossentropy", optimizer=opt,
                  metrics=["accuracy"])
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-5)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

In [ ]:
# construct the set of callbacks
callbacks = [EpochCheckpoint(output_path,
                             every=5,
                             startAt=start_epoch),
             TrainingMonitor(config.fig_path,
                             jsonPath=config.json_path,
                             startAt=start_epoch)]